# 1. Import libraries

In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import src.geo_functions as gf
from bson.json_util import dumps
import json
import pandas as pd

# Importing the madrid functions:
import src.geo_functions as gf

# 2. Make MongoDB Connection

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("geospatial")

In [3]:
lisbon = db.get_collection("lisbon")
london = db.get_collection("london")
madrid = db.get_collection("madrid")

# 3. Indexes creation

In [ ]:
db.lisbon.create_index([("location", GEOSPHERE)])
db.london.create_index([("location", GEOSPHERE)])
db.madrid.create_index([("location", GEOSPHERE)])

# 4. Some extra cleanning

In [4]:
diccionario = {"Pub": "Bar",
            "Metro": 'Train Station',
            "Café": "Bar",
            "Bar": "Bar",
            "Comedy Club": "Bar",
            "Steakhouse": "Bar",
            "Hotel Bar": "Bar",
            "Cocktail":"Bar",
            "Lounge": "Bar",
            "Nightclub": "Bar",
            "Restaurant": "Bar",
            "Sports Bar": "Bar", 
            "Bistro": "Bar",
            "Wine Bar": "Bar",
            "Pizza": "Bar",  
            'Bus Stop': 'Train Station',
            "Nightclub": "Bar",
            "Spanish": 'Bar',
            "Winery": "Bar",
            "Cocktail": "Bar",
            "Tapas": "Bar",
            "Coffee Shop": "Starbucks", 
            "Japanese": "Bar",
            "Seafood": "Bar",  
            "Brewery":"Bar",
            "Diner": "Bar",
            "Breakfast": "Bar",
            "Chinese": "Bar",
            'New American': 'Bar',
            "Cafeteria": "Bar",
            "Salon / Barbershop": "Bar",
            "Roof Deck": "Bar",
            "Sushi": "Bar", 
            "Snacks": "Bar",  
            "Karaoke":"Bar",
            "Corporate Cafeteria": "Bar",
            "Ice Cream": "Bar",
            'Portuguese': 'Bar',
            'Train Station': 'Train Station'}

# 5. Geoqueries

## LONDON 

In [5]:
london_ = gf.geocode("St Paul´s Cathedral, London")

In [6]:
query_london = [{"$geoNear" : {'near': [51.51421, -0.05688],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

geoloc_london = london.aggregate(query_london)
london_json = json.loads(dumps(geoloc_london))

In [7]:
# Now we create a dataframe out of the json obtained:

In [8]:
london_df = pd.DataFrame(london_json)

In [9]:
gf.drop_columns(london_df)

In [10]:
london_df["city"] = "London"

In [11]:
london_df['shortname'] = london_df['shortname'].map(diccionario)

In [12]:
london_df.dropna(how='any', axis=0, inplace=True)

In [13]:
london_df['shortname'].unique()

array(['Train Station', 'Bar', 'Starbucks'], dtype=object)

## LISBON

In [14]:
lisbon_ = gf.geocode("Avenida Antonio Augusto de Aguiar, Lisboa")

In [15]:
query_lisbon = [{"$geoNear" : {'near': [38.73280, -9.15221],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

geoloc_lisbon = lisbon.aggregate(query_lisbon)
lisbon_json = json.loads(dumps(geoloc_lisbon))

In [16]:
# Now we create a dataframe out of the json obtained:

In [17]:
lisbon_df = pd.DataFrame(lisbon_json)

In [18]:
gf.drop_columns(lisbon_df)

In [19]:
lisbon_df["city"] = "Lisbon"

In [20]:
lisbon_df['shortname'] = lisbon_df['shortname'].map(diccionario)

## MADRID

In [21]:
madrid_ = gf.geocode("Paseo de la Castellana, Madrid")

In [22]:
query_madrid = [{"$geoNear" : {'near': [40.46879, -3.68915],
                        'distanceField' : 'distance',
                        'maxDistance' : 1000,
                        'distanceMultiplier': 6371,
                        'spherical' : True}}]

geoloc_madrid = madrid.aggregate(query_madrid)
madrid_json = json.loads(dumps(geoloc_madrid))

In [23]:
# Now we create a dataframe out of the json obtained:

In [24]:
madrid_df = pd.DataFrame(madrid_json)

In [25]:
gf.drop_columns(madrid_df)

In [26]:
madrid_df["city"] = "Madrid"

In [27]:
madrid_df['shortname'] = madrid_df['shortname'].map(diccionario)

In [28]:
madrid_df.dropna(how='any', axis=0, inplace=True)

In [29]:
madrid_df['shortname'].unique()

array(['Bar', 'Train Station', 'Starbucks'], dtype=object)

# 6. DATA FOR PONDERATIONS

In [30]:
# First, we create a new dataframe with the information of the three cities:

In [41]:
data = madrid_df.append([lisbon_df, london_df])

In [42]:
data

,name,latitud,longitud,shortname,distance,city
1,"Bar ""La mina"" Café",40.469347,-3.687093,Bar,0.236913,Madrid
2,Bar Machico,40.469166,-3.686506,Bar,0.296912,Madrid
3,Bar J. Simón,40.469461,-3.686426,Bar,0.311912,Madrid
4,Dala's Cervecería Bar,40.469888,-3.686491,Bar,0.319800,Madrid
6,Fun and English Bar,40.465854,-3.689926,Bar,0.337027,Madrid
...,...,...,...,...,...,...
83,The Coffee Bar,51.499171,-0.162803,Starbucks,11.896263,London
84,Starbucks,51.518920,-0.168813,Starbucks,12.457396,London
85,London Paddington Railway Station (PAD) (Londo...,51.516754,-0.176843,Train Station,13.342316,London
86,Clapham Junction Railway Station (CLJ),51.464185,-0.170416,Train Station,13.795807,London


In [43]:
# Second, we apply the ponderation funcions:

In [44]:
data["Puntuation"] = data["shortname"].apply(gf.puntuacion)

In [45]:
# Third, we create a new column "Total":

In [46]:
data["Total"] = data["distance"]*data["Puntuation"]

In [47]:
# Fourth, we create a new dataframe where we can see the result:

In [48]:
final_punctuations = pd.DataFrame(data.groupby(['city'])["Total"].sum())

In [49]:
final_punctuations

,Total
city,
Lisbon,155.327020
London,131.877389
Madrid,79.574407


As we can see in the latter table, Lisbon will be the best city to open the company, as it fulfils the most requirements.

In [40]:
gf.normalizacion(final_punctuations, "Total")

[1.0, 0.6904445949273891, 0.0]

# 7. Visualization

In [ ]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
madrid_

In [ ]:
map_1 = folium.Map(location=[40.46879,-3.68915], zoom_start=15)
map_1;

In [ ]:
for i,row in madrid_df.iterrows():

    dic = {"location": [row["latitud"], row["longitud"]], "tooltip": row["name"]}

    if row["shortname"] == "Starbucks":
        icono = Icon(color = "red",
                     prefix="fa",
                     icon="laptop",
                     icon_color="black")

    elif row["shortname"] == 'Train Station':
        icono = Icon(color = "green",
                     prefix="fa",
                     icon="glass",
                     icon_color="black")

        
    elif row["shortname"] == "Bar":        
        icono = Icon(color = "cadetblue",
                     prefix="fa",
                     icon="bug",
                     icon_color="black")
        

    mark = Marker(**dic, icono=Icon)
    mark.add_to(map_1)   
    
map_1;